# Week 3 - Homework for the course Make Money with Machine Learning from Siraj Raval

The Question:

In this competition you are predicting the probability that an online transaction is fraudulent, as denoted by the binary target isFraud.


## Imports

In [ ]:
import json
import zipfile
import os, sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn

## Download the dataset from kaggle



In [ ]:
# to turn on and off visualization because of the time consumption
make_visualization = True

username = ""  # insert your username here to download the kaggle dataset
key = "" # insert your API key here to download the kaggle dataset 

# Values for testing submission_v3.csv - this was better
# Score: 0.7033

#(array([0.9768539 , 0.76453565]),
# array([0.99607336, 0.35073321]),
# array([0.98637002, 0.48086708]),
# array([455861,  16571]))

submission_name = "submission_v3.csv"
nan_limit = 70
max_iter=500
test_size=0.8
solver='lbfgs'

# Values for submission_v1.csv
#submission_name = "submission_v1.csv"
#test_size=0.6
#nan_limit = 80
#max_iter=200
#solver='lbfgs'

# Values for testing submission_v2.csv - it was not better
#submission_name = "submission_v2.csv"
#nan_limit = 60
#test_size=0.4
#max_iter=200
#solver='lbfgs'

#(array([0.97632943, 0.81419624]),
# array([0.99726723, 0.33123028]),
# array([0.98668727, 0.47089263]),
# array([227974,   8242]))

# try another solver #
# Score: 0.6543
#submission_name = "submission_v4.csv"
#nan_limit = 70
#max_iter=500
#test_size=0.8
#solver='saga'

# more iterations
# Score: 
#submission_name = "submission_v5.csv"
#nan_limit = 70
#max_iter=1000
#test_size=0.8
#solver='lbfgs'

In [ ]:
!pip install kaggle

api_token = {"username": username,"key": key}

!mkdir /root/.kaggle
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json # that nobody else can read your file

In [ ]:
# sometimes one must start 2 times in colab - I don't know why 
!kaggle competitions download -c ieee-fraud-detection
print(os.listdir())


## Some helper functions to handle numerical and categorical data

In [ ]:
def create_col_name(base_str, start_int, end_int):
    return [base_str + str(i) for i in range(start_int, end_int+1)]

In [ ]:
create_col_name('card', 1, 6)

In [ ]:
cat_cols = (['ProductCD'] + create_col_name('card', 1, 6) + ['addr1', 'addr2', 'P_emaildomain', 'R_emaildomain'] + 
            create_col_name('M', 1, 9) + ['DeviceType', 'DeviceInfo'] + create_col_name('id_', 12, 38))

id_cols = ['TransactionID', 'TransactionDT']

target = 'isFraud'

In [ ]:
type_map = {c: str for c in cat_cols + id_cols}

## Load the training data

In [ ]:
os.chdir('/content')
df_train_id = pd.read_csv('train_identity.csv.zip', dtype=type_map)
df_train_trans = pd.read_csv('train_transaction.csv.zip', dtype=type_map)

In [ ]:
df_train_id.shape, df_train_trans.shape

In [ ]:
#df_train_trans.dtypes

## Visualize the dataset - 1

In [ ]:
if make_visualization:
  df_train_trans.head(10)

In [ ]:
if make_visualization:
  df_train_id.head()

## Merge the data

In [ ]:
df_train = df_train_trans.merge(df_train_id, on='TransactionID', how='left')

In [ ]:
numeric_cols = [col for col in df_train.columns.tolist() if col not in cat_cols + id_cols + [target]]

In [ ]:
assert(df_train.shape[0]==df_train_trans.shape[0])

In [ ]:
if make_visualization:
  df_train.head(10)

In [ ]:
if make_visualization: 
  df_train[cat_cols].head()

In [ ]:
if make_visualization: 
  df_train[numeric_cols].head()

In [ ]:
df_train.info()

In [ ]:
#df_drop = df_train.dropna()

In [ ]:
# yeah that's great - no data remained !!!! 
#df_drop.head()

## Remove columns with NAN over a certain limit

In [ ]:
# ok let us try another method 
# remove the columns with NAN over a certain limit
nan_limit = 80

In [ ]:
# processing categorical values
cat_cols_missing = {}
cat_cols_good = []
cat_cols_bad = []

for col in cat_cols:
  cat_cols_missing[col] = 100*df_train[col].isnull().sum()/len(df_train[col])
  #print(col + ': ' + str(100*df_train[col].isnull().sum()/len(df_train[col])))
  if cat_cols_missing[col] < nan_limit:
    cat_cols_good.append(col)
  else:
    cat_cols_bad.append(col)

In [ ]:
# processing numerical values
numeric_cols_missing = {}
numeric_cols_good = []
numeric_cols_bad = []
# isFraud is always good
#numeric_cols_good.append(target)
for col in numeric_cols:
  numeric_cols_missing[col] = 100*df_train[col].isnull().sum()/len(df_train[col])
  #print(col + ': ' + str(100*df_train[col].isnull().sum()/len(df_train[col])))
  if numeric_cols_missing[col] < nan_limit:
    numeric_cols_good.append(col)
  else:
    numeric_cols_bad.append(col)

In [ ]:
numeric_cols_good
good_cols = []
good_cols.extend(numeric_cols_good)
good_cols.extend(cat_cols_good)
good_cols.append(target)

bad_cols = []
bad_cols.extend(numeric_cols_bad)
bad_cols.extend(cat_cols_bad)

all_cols = []
all_cols.extend(numeric_cols)
all_cols.extend(cat_cols)

In [ ]:
#all_cols, good_cols, bad_cols

In [ ]:
# here I can optimize the code !!!
df_try1 = pd.DataFrame()
df_try2 = pd.DataFrame()
numeric_cols_good2 = numeric_cols_good[:]
numeric_cols_good2.append("isFraud")
df_try1 = df_train[numeric_cols_good2]
df_try2 = df_train[cat_cols_good]

df_try = df_try1.merge(df_try2, left_index=True, right_index=True, how='left')

df_try1.shape, df_try2.shape, df_try.shape
#df_try1.append(df_try2, ignore_index=True)


In [ ]:
df_train[numeric_cols_good].shape, df_train[cat_cols_good].shape, df_train['isFraud'].shape

In [ ]:
assert(df_train.shape[0]==df_try.shape[0])

In [ ]:
if make_visualization: 
  df_try.head(10)

In [ ]:
df_try.shape, df_train.shape

In [ ]:
df_try.info()

In [ ]:
# How many isFraud data did we have?
df_try['isFraud'].sum()

In [ ]:
100-df_try['isFraud'].sum()/df_try.shape[0]*100

Questions:
- How to deal with the categorical data - made a pipeline for both numerical and categorical data
- How to deal with the unbalance of the dataset (much more nonFraud than isFraud) - at the moment I don't treat it :-)
- How to deal with the NAN's for logistic regression models (can I treat them as zeros?) - in the pipeline I treat them as 'missing'

## Visualization of the dataset - 2

In [ ]:
import seaborn as sns

In [ ]:
if make_visualization: 
  sns.set(style="whitegrid", context="notebook")
  cols = numeric_cols[0:10]
  cols.append('isFraud')
  sns.pairplot(df_train[cols], size=2.5)
  plt.show()

In [ ]:
if make_visualization: 
  sns.set(style="whitegrid", context="notebook")
  cols = numeric_cols[11:21]
  cols.append('isFraud')
  sns.pairplot(df_train[cols], size=2.5)
  plt.show()

In [ ]:
if make_visualization: 
  sns.set(style="whitegrid")
  ax = sns.stripplot(x="isFraud", y="card1", data=df_train)

In [ ]:
if make_visualization: 
  sns.set(style="whitegrid")
  ax = sns.stripplot(x="isFraud", y="dist1", data=df_train)

In [ ]:
if make_visualization:
  ax = sns.stripplot(x="card1", y="dist1", hue="isFraud", data=df_train, jitter=True)














## Train a model

In [ ]:
# time for machine learning
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

In [ ]:
# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = numeric_cols_good
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

In [ ]:
categorical_features = cat_cols_good
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(C=1e5, solver=solver, max_iter=max_iter))])
#clf = Pipeline(steps=[('preprocessor', preprocessor),
#                      ('classifier', LogisticRegression(C=1e5, solver='saga', max_iter=200))])

In [ ]:
x = df_try.drop('isFraud', axis=1)
y = df_try['isFraud']

#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.6)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size)



In [ ]:
clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

In [ ]:
X_train.head()

In [ ]:
100-100*y_train.sum()/len(y_train), y_train.sum(), len(y_train), 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
predicted_values = clf.predict(X_test)

In [ ]:
precision_recall_fscore_support(y_test,predicted_values)
# precision, recall, F-measure, support 

## Load test data

In [ ]:
df_test_id = pd.read_csv('test_identity.csv.zip', dtype=type_map)
df_test_trans = pd.read_csv('test_transaction.csv.zip', dtype=type_map)

In [ ]:
df_test_id.head()

In [ ]:
df_test_trans.head(10)

In [ ]:
df_test = df_test_trans.merge(df_test_id, on='TransactionID', how='left')

In [ ]:
df_test.head()

In [ ]:
bad_cols.extend(id_cols)

In [ ]:
x_verification = df_test.drop(bad_cols, axis=1)

## Predict the values for the test data

In [ ]:
df_test_values = clf.predict(x_verification)

## Save a File for Kaggle competition

In [ ]:
df_test["isFraud"] = df_test_values

In [ ]:
df_test[['TransactionID', 'isFraud']].to_csv(submission_name, index=False)

In [ ]:
 ax = sns.stripplot(x=df_test["isFraud"])

In [ ]:
df_test.head()